# Step 1: Pascal VOC (XML) → YOLO (TXT) 변환

Dataset 1 (kaggle-safey_helmet)의 XML 라벨을 YOLO 형식으로 변환합니다.

## Pascal VOC 형식이란?
- XML 기반의 객체 탐지 라벨 형식
- 바운딩 박스를 절대 좌표(픽셀)로 표현: xmin, ymin, xmax, ymax
- 각 이미지마다 하나의 XML 파일

## YOLO 형식이란?
- TXT 기반의 객체 탐지 라벨 형식
- 바운딩 박스를 정규화된 상대 좌표(0~1)로 표현: x_center, y_center, width, height
- 각 이미지마다 하나의 TXT 파일
- 한 줄에 하나의 객체: class_id x_center y_center width height

## 변환 공식
- x_center = (xmin + xmax) / 2 / image_width
- y_center = (ymin + ymax) / 2 / image_height
- width = (xmax - xmin) / image_width
- height = (ymax - ymin) / image_height

## 클래스 매핑
- helmet → 0 (사용)
- head → -1 (제외: 미착용 상태 탐지 불필요)
- person → -1 (제외: 전신 탐지 불필요)

## 입력/출력
- 입력: dataset/raw_data/raw/kaggle-safey_helmet/annotations/*.xml
- 출력: dataset/raw_data/processed/dataset1/labels/*.txt

In [ ]:
import xml.etree.ElementTree as ET
import os
from pathlib import Path
import shutil

## 클래스 매핑 정의

Dataset 1의 원본 클래스를 우리 프로젝트의 클래스 ID로 매핑합니다.
-1은 해당 클래스를 제외한다는 의미입니다.

In [ ]:
# 클래스 매핑 정의
CLASS_MAPPING = {
    'helmet': 0,   # 헬멧 착용 → class 0
    'head': -1,    # 헬멧 미착용 (머리만) → 제외
    'person': -1   # 전신 → 제외
}

## 변환 함수 정의

단일 Pascal VOC XML 파일을 YOLO TXT 형식으로 변환하는 함수입니다.

In [ ]:
def convert_voc_to_yolo(xml_file: str, output_dir: str) -> bool:
    """
    단일 Pascal VOC XML 파일을 YOLO TXT 형식으로 변환
    
    Args:
        xml_file: 변환할 XML 파일의 전체 경로
        output_dir: 변환된 TXT 파일을 저장할 디렉토리
    
    Returns:
        bool: 변환 성공 여부
    """
    try:
        # 1. XML 파일 파싱
        tree = ET.parse(xml_file)
        root = tree.getroot()
        
        # 2. 이미지 크기 정보 추출
        size = root.find('size')
        img_width = int(size.find('width').text)
        img_height = int(size.find('height').text)
        
        # 출력 파일 경로 설정
        filename = Path(xml_file).stem + '.txt'
        output_path = os.path.join(output_dir, filename)
        
        lines = []
        
        # 3. 모든 객체(object) 처리
        for obj in root.findall('object'):
            class_name = obj.find('name').text
            
            if class_name not in CLASS_MAPPING:
                continue
            
            class_id = CLASS_MAPPING[class_name]
            
            if class_id == -1:
                continue
            
            # 바운딩 박스 좌표 추출
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)
            
            # YOLO 형식으로 변환 (정규화된 중심점 + 크기)
            x_center = (xmin + xmax) / 2.0 / img_width
            y_center = (ymin + ymax) / 2.0 / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height
            
            lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
        
        # 4. 파일 저장
        if lines:
            with open(output_path, 'w') as f:
                f.write('\n'.join(lines))
            return True
        
        return False
    
    except Exception as e:
        print(f"❌ 변환 실패: {xml_file} - {str(e)}")
        return False

## 경로 설정

In [ ]:
# 프로젝트 루트 디렉토리 (notebooks/preprocess 폴더 기준)
base_dir = Path.cwd().parent.parent

# 입력 경로
input_dir = base_dir / 'dataset' / 'raw_data' / 'raw' / 'kaggle-safey_helmet'
annotations_dir = input_dir / 'annotations'
images_dir = input_dir / 'images'

# 출력 경로
output_dir = base_dir / 'dataset' / 'raw_data' / 'processed' / 'dataset1'
output_images_dir = output_dir / 'images'
output_labels_dir = output_dir / 'labels'

print(f"입력 경로: {input_dir}")
print(f"출력 경로: {output_dir}")

## 출력 디렉토리 생성

In [ ]:
output_images_dir.mkdir(parents=True, exist_ok=True)
output_labels_dir.mkdir(parents=True, exist_ok=True)
print("✅ 출력 디렉토리 생성 완료")

## 변환 실행

In [ ]:
# XML 파일 목록
xml_files = list(annotations_dir.glob('*.xml'))
total = len(xml_files)

print(f"총 {total}개 파일 변환 중...")

converted = 0
skipped = 0

for i, xml_file in enumerate(xml_files, 1):
    if convert_voc_to_yolo(str(xml_file), str(output_labels_dir)):
        # 이미지 복사
        image_name = xml_file.stem + '.png'
        src_image = images_dir / image_name
        dst_image = output_images_dir / image_name
        
        if src_image.exists():
            shutil.copy(src_image, dst_image)
            converted += 1
        else:
            for ext in ['.jpg', '.jpeg']:
                alt_image = images_dir / (xml_file.stem + ext)
                if alt_image.exists():
                    shutil.copy(alt_image, output_images_dir / alt_image.name)
                    converted += 1
                    break
    else:
        skipped += 1
    
    if i % 500 == 0 or i == total:
        print(f"  진행: {i}/{total} ({i*100//total}%)")

## 결과 확인

In [ ]:
print()
print("=" * 50)
print("✅ Step 1: 변환 완료!")
print("=" * 50)
print(f"   - 변환됨: {converted}개")
print(f"   - 스킵됨 (helmet 없음): {skipped}개")
print(f"   - 출력 위치: {output_dir}")

## 변환 결과 샘플 확인

In [ ]:
# 변환된 라벨 파일 하나 확인
sample_labels = list(output_labels_dir.glob('*.txt'))[:3]
for lbl in sample_labels:
    print(f"\n📄 {lbl.name}:")
    with open(lbl, 'r') as f:
        print(f.read())